In [15]:
import pandas as pd
import spacy
import pickle
import os
import re
from datetime import datetime
os.getcwd()

'C:\\Users\\camil\\Documents\\school\\ADS\\Personalisation for public media\\Assignment 2'

In [2]:
#Read in all csvs

df_2019 = pd.read_csv('articles_2019.csv', sep="|", quotechar="'" , on_bad_lines='skip')
df_2020 = pd.read_csv('articles_2020.csv', sep="|", quotechar="'" , on_bad_lines='skip')
df_2021 = pd.read_csv('articles_2021.csv', sep="|", quotechar="'" , on_bad_lines='skip')
df_2022 = pd.read_csv('articles_2022.csv', sep="|", quotechar="'" , on_bad_lines='skip')
df_2023 = pd.read_csv('articles_2023.csv', sep="|", quotechar="'" , on_bad_lines='skip')
df_2023

,url,title,article,date,tags,image;;
0,https://nos.nl/artikel/2458417-nederlandse-28-...,Nederlandse (28) komt om bij skiongeluk Oosten...,In Oostenrijk is een 28-jarige vrouw uit Neder...,2023-01-01,['Buitenland'],https://cdn.nos.nl/image/2023/01/01/930609/102...
1,"""https://nos.nl/artikel/2458410-washington-wil...",Washington wil van overschot aan lege kantoorp...,"""""Grote Amerikaanse binnensteden ondergaan sin...",2023-01-01,['Buitenland'],https://cdn.nos.nl/image/2023/01/01/930568/102...
2,"""https://nos.nl/artikel/2458405-lula-beedigd-a...",Lula beëdigd als nieuwe president van Brazilië...,"""""In het parlement in de Braziliaanse hoofdsta...",2023-01-01,['Buitenland'],https://cdn.nos.nl/image/2023/01/01/930614/102...
3,"""https://nos.nl/artikel/2458396-vrt-vindt-verl...",VRT vindt verloren interview met bedenker oerk...,"""""De Vlaamse publieke omroep VRT heeft een ver...",2023-01-01,['Buitenland'],https://cdn.nos.nl/image/2023/01/01/930537/102...
4,"""https://nos.nl/artikel/2458390-chinese-leger-...",Chinese leger haalt uit naar VS om 'hypen' str...,"""""Vanuit Peking is fel gereageerd op beschuldi...",2023-01-01,['Buitenland'],https://cdn.nos.nl/image/2023/01/01/930532/102...
...,...,...,...,...,...,...
1025,"""https://nos.nl/artikel/2467536-james-webb-tel...",James Webb-telescoop legt zeldzaam moment ster...,"""""Op een nieuwe, gedetailleerde foto van de Ja...",2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951872/102...
1026,https://nos.nl/artikel/2467529-minstens-vijf-d...,Minstens vijf doden door overstromingen in aar...,In Turkije zijn minstens tien doden gevallen d...,2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951932/102...
1027,"""https://nos.nl/artikel/2467526-politie-pakist...",Politie Pakistan weer slaags met aanhangers bi...,"""""Voor de tweede dag op een rij zijn er hevige...",2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951853/102...
1028,https://nos.nl/artikel/2467504-wekdienst-15-3-...,Wekdienst 15/3: Stemmen voor Provinciale State...,Goedemorgen! Ruim 13 miljoen Nederlandse kieze...,2023-03-15,"['Binnenland', 'Buitenland']",https://cdn.nos.nl/image/2023/03/15/951807/102...


In [3]:
#Make sure there are no parentheses
df_2019['article'] = df_2019['article'].apply(lambda row : row.strip('\"'))
df_2020['article'] = df_2020['article'].apply(lambda row : row.strip('\"'))
df_2021['article'] = df_2021['article'].apply(lambda row : row.strip('\"'))
df_2022['article'] = df_2022['article'].apply(lambda row : row.strip('\"'))
df_2023['article'] = df_2023['article'].apply(lambda row : row.strip('\"'))

df_2019['url'] = df_2019['url'].apply(lambda row : row.strip('\"'))
df_2020['url'] = df_2020['url'].apply(lambda row : row.strip('\"'))
df_2021['url'] = df_2021['url'].apply(lambda row : row.strip('\"'))
df_2022['url'] = df_2022['url'].apply(lambda row : row.strip('\"'))
df_2023['url'] = df_2023['url'].apply(lambda row : row.strip('\"'))

In [4]:
#Delete rows that violate column structure
#This happens only 6 times, for articles that have '|' in their titles
df_2019 = df_2019.dropna()
df_2020 = df_2020.dropna()
df_2021 = df_2021.dropna()
df_2022 = df_2022.dropna()
df_2023 = df_2023.dropna()

In [5]:
#Reindex dfs so that concatination goes well

index2020start = len(df_2019.index)
index2021start = index2020start + len(df_2020.index)
index2022start = index2021start + len(df_2021.index)
index2023start = index2022start + len(df_2022.index)
index2023end = index2023start + len(df_2023.index)

index2019 = [i for i in range(0, len(df_2019.index))]
index2020 = [i for i in range(index2020start, index2021start)]
index2021 = [i for i in range(index2021start, index2022start)]
index2022 = [i for i in range(index2022start, index2023start)]
index2023 = [i for i in range(index2023start, index2023end)]

df_2019.index = index2019
df_2020.index = index2020
df_2021.index = index2021
df_2022.index = index2022
df_2023.index = index2023
df_2023

,url,title,article,date,tags,image;;
21564,https://nos.nl/artikel/2458417-nederlandse-28-...,Nederlandse (28) komt om bij skiongeluk Oosten...,In Oostenrijk is een 28-jarige vrouw uit Neder...,2023-01-01,['Buitenland'],https://cdn.nos.nl/image/2023/01/01/930609/102...
21565,https://nos.nl/artikel/2458410-washington-wil-...,Washington wil van overschot aan lege kantoorp...,Grote Amerikaanse binnensteden ondergaan sinds...,2023-01-01,['Buitenland'],https://cdn.nos.nl/image/2023/01/01/930568/102...
21566,https://nos.nl/artikel/2458405-lula-beedigd-al...,Lula beëdigd als nieuwe president van Brazilië...,In het parlement in de Braziliaanse hoofdstad ...,2023-01-01,['Buitenland'],https://cdn.nos.nl/image/2023/01/01/930614/102...
21567,https://nos.nl/artikel/2458396-vrt-vindt-verlo...,VRT vindt verloren interview met bedenker oerk...,De Vlaamse publieke omroep VRT heeft een verlo...,2023-01-01,['Buitenland'],https://cdn.nos.nl/image/2023/01/01/930537/102...
21568,https://nos.nl/artikel/2458390-chinese-leger-h...,Chinese leger haalt uit naar VS om 'hypen' str...,Vanuit Peking is fel gereageerd op beschuldigi...,2023-01-01,['Buitenland'],https://cdn.nos.nl/image/2023/01/01/930532/102...
...,...,...,...,...,...,...
22581,https://nos.nl/artikel/2467536-james-webb-tele...,James Webb-telescoop legt zeldzaam moment ster...,"Op een nieuwe, gedetailleerde foto van de Jame...",2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951872/102...
22582,https://nos.nl/artikel/2467529-minstens-vijf-d...,Minstens vijf doden door overstromingen in aar...,In Turkije zijn minstens tien doden gevallen d...,2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951932/102...
22583,https://nos.nl/artikel/2467526-politie-pakista...,Politie Pakistan weer slaags met aanhangers bi...,Voor de tweede dag op een rij zijn er hevige o...,2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951853/102...
22584,https://nos.nl/artikel/2467504-wekdienst-15-3-...,Wekdienst 15/3: Stemmen voor Provinciale State...,Goedemorgen! Ruim 13 miljoen Nederlandse kieze...,2023-03-15,"['Binnenland', 'Buitenland']",https://cdn.nos.nl/image/2023/03/15/951807/102...


In [6]:
#Rename columns and combine dfs

df_2019.columns = ["url", "title", "article", "date", "tags", "image"]
df_2020.columns = ["url", "title", "article", "date", "tags", "image"]
df_2021.columns = ["url", "title", "article", "date", "tags", "image"]
df_2022.columns = ["url", "title", "article", "date", "tags", "image"]
df_2023.columns = ["url", "title", "article", "date", "tags", "image"]

articles = pd.concat([df_2019, df_2020])
articles = pd.concat([articles, df_2021])
articles = pd.concat([articles, df_2022])
articles = pd.concat([articles, df_2023])
articles

,url,title,article,date,tags,image
0,https://nos.nl/artikel/2265855-gewapende-manne...,Gewapende mannen doden 37 burgers in Mali,"In het midden van Mali, in de Mopti-regio, heb...",2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522391/xxl...
1,https://nos.nl/artikel/2265853-migranten-ontde...,Migranten ontdekt in matrassen aan Spaanse grens,De Spaanse politie heeft afgelopen weekend twe...,2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522387/xxl...
2,https://nos.nl/artikel/2265844-sea-watch-ziet-...,Sea-Watch ziet situatie op vluchtelingenschip ...,De situatie aan boord van het vluchtelingensch...,2019-01-01,"['Binnenland', 'Buitenland']",https://cdn.nos.nl/image/2018/12/28/521681/xxl...
3,https://nos.nl/artikel/2265833-bolsonaro-brazi...,Bolsonaro: Brazilië nu bevrijd van socialisme ...,In de Braziliaanse hoofdstad Brasilia is de ui...,2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522367/xxl...
4,https://nos.nl/artikel/2265824-ruimtesonde-new...,Ruimtesonde New Horizons veilig langs ruimtero...,NASA heeft een nieuw signaal binnengekregen va...,2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522361/xxl...
...,...,...,...,...,...,...
22581,https://nos.nl/artikel/2467536-james-webb-tele...,James Webb-telescoop legt zeldzaam moment ster...,"Op een nieuwe, gedetailleerde foto van de Jame...",2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951872/102...
22582,https://nos.nl/artikel/2467529-minstens-vijf-d...,Minstens vijf doden door overstromingen in aar...,In Turkije zijn minstens tien doden gevallen d...,2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951932/102...
22583,https://nos.nl/artikel/2467526-politie-pakista...,Politie Pakistan weer slaags met aanhangers bi...,Voor de tweede dag op een rij zijn er hevige o...,2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951853/102...
22584,https://nos.nl/artikel/2467504-wekdienst-15-3-...,Wekdienst 15/3: Stemmen voor Provinciale State...,Goedemorgen! Ruim 13 miljoen Nederlandse kieze...,2023-03-15,"['Binnenland', 'Buitenland']",https://cdn.nos.nl/image/2023/03/15/951807/102...


In [7]:
#Clean image urls
articles['image'] = articles['image'].apply(lambda row : str(row))
articles['image'] = articles['image'].apply(lambda row : row.strip(';"'))

#Some enters in articles translated as """", fix that with regex:
articles['article'] = articles['article'].apply(lambda row : re.sub('""""', '"', row))
articles['title'] = articles['title'].apply(lambda row : re.sub('""""', '"', row))

In [8]:
nlp = spacy.load("nl_core_news_sm")

#Add more stop words
custom_stops = {"gaan", "gaat", "ga", "ging", "gingen", "gegaan",
                "maken", "maakt", "maak", "maakte", "maakten", "gemaakt",
                "zeggen", "zegt", "zeg", "zei", "zeiden", "gezegd",
                "komen", "kom", "komt", "kwam", "kwamen", "gekomen",
                "zien", "ziet", "zie", "zag", "zagen", "gezien",
                "staan", "staat", "sta", "stond", "stonden", "gestaan",
               "zo'n", "ergens", "jaar", "dag", "week", "vorig", "uur",
               "maand", "twee", "drie", "vier", "vijf", "zes", "zeven", "acht",
               "negen", "tien", "twintig", "dertig", "veertig", "vijftig",
               "zestig", "zeventig", "tachtig", "negentig", "honderd"
               "duizend", "miljoen"}

nlp.Defaults.stop_words |= custom_stops
print(nlp.Defaults.stop_words)

#Process all paragraphs using spacy
processed_articles = [text for text in (nlp.pipe(articles['article'], 
                                            n_process=4, # four threads
                                            disable=["ner",
                                                    "parser"]))]
processed_titles = [text for text in (nlp.pipe(articles['title'], 
                                            n_process=4, # four threads
                                            disable=["ner",
                                                    "parser"]))]

{'dezen', 'en', 'al', 'uitgezonderd', 'mocht', 'omstreeks', 'alleen', 'daarnet', 'dikwijls', 'prof', 'ter', 'bij', 'mijn', 'door', 'overigens', 'spoedig', 'klaar', 'wiens', 'gauw', 'anders', 'inmiddels', 'wanneer', 'zonder', 'jouw', 'alhoewel', 'juist', 'je', 'ge', 'maakt', 'daarna', 'naar', 'verder', 'zouden', 'maakte', 'veeleer', 'maand', 'bovendien', 'gingen', 'bijna', 'hun', 'gemaakt', 'gegaan', 'hebben', 'kwam', 'met', 'iets', 'totdat', 'wilde', 'wij', 'enkel', 'af', 'u', 'steeds', 'vooruit', 'staan', 'doorgaans', 'geworden', "'t", 'bent', 'komen', 'vijftig', 'welk', 'zulke', 'gekomen', 'hare', 'in', 'reeds', 'allen', 'ook', 'drie', 'gelijk', 'waren', 'eerste', 'afgelopen', 'dat', 'zagen', 'waar', 'anderen', 'wil', 'mijzelf', 'wel', 'nogal', 'boven', 'kon', 'voorts', 'voordat', 'thans', 'want', 'moeten', 'aangaangde', 'weg', 'twee', 'gewoon', 'betere', 'gedurende', 'allebei', 'eveneens', 'er', 'voor', 'worden', 'jijzelf', 'publ', 'later', 'omhoog', 'ieder', 'omlaag', 'zijn', 'ande

In [9]:
#Clean all texts and tokenize for LDA and word embeddings
cleaned_articles = [[word.lemma_.lower() for word in processed_article if 
                                                                    not word.is_stop and # remove stopwords
                                                                    not word.is_punct and # remove punctuation tokens
                                                                    not word.is_digit and # remove numbers 
                                                                    not word.like_url and # remove URLs
                                                                    not word.is_space] # remove trail spaces
                            for processed_article in processed_articles]

cleaned_titles = [[word.lemma_.lower() for word in processed_title if 
                                                                    not word.is_stop and # remove stopwords
                                                                    not word.is_punct and # remove punctuation tokens
                                                                    not word.is_digit and # remove numbers 
                                                                    not word.like_url and # remove URLs
                                                                    not word.is_space] # remove trail spaces
                            for processed_title in processed_titles]

In [10]:
#Add cleaned columns to df
articles['cleaned_article'] = cleaned_articles
articles['cleaned_title'] = cleaned_titles
articles

,url,title,article,date,tags,image,cleaned_article,cleaned_title
0,https://nos.nl/artikel/2265855-gewapende-manne...,Gewapende mannen doden 37 burgers in Mali,"In het midden van Mali, in de Mopti-regio, heb...",2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522391/xxl...,"[midden, mali, mopti-regio, wapenen, man, burg...","[gewapend, man, dood, burger, mali]"
1,https://nos.nl/artikel/2265853-migranten-ontde...,Migranten ontdekt in matrassen aan Spaanse grens,De Spaanse politie heeft afgelopen weekend twe...,2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522387/xxl...,"[spaans, politie, weekend, afrikaans, migrant,...","[migranten, ontdekken, matrasis, spaans, grens]"
2,https://nos.nl/artikel/2265844-sea-watch-ziet-...,Sea-Watch ziet situatie op vluchtelingenschip ...,De situatie aan boord van het vluchtelingensch...,2019-01-01,"['Binnenland', 'Buitenland']",https://cdn.nos.nl/image/2018/12/28/521681/xxl...,"[situatie, boord, vluchtelingenschip, sea-watc...","[sea-watch, situatie, vluchtelingenschip, nijpen]"
3,https://nos.nl/artikel/2265833-bolsonaro-brazi...,Bolsonaro: Brazilië nu bevrijd van socialisme ...,In de Braziliaanse hoofdstad Brasilia is de ui...,2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522367/xxl...,"[braziliaans, hoofdstad, brasilia, uiterst, re...","[bolsonaro, brazilië, bevrijden, socialisme, p..."
4,https://nos.nl/artikel/2265824-ruimtesonde-new...,Ruimtesonde New Horizons veilig langs ruimtero...,NASA heeft een nieuw signaal binnengekregen va...,2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522361/xxl...,"[nasa, nieuw, signaal, binnengekrijgen, ruimte...","[ruimtesonde, new, horizons, veilig, langs, ru..."
...,...,...,...,...,...,...,...,...
22581,https://nos.nl/artikel/2467536-james-webb-tele...,James Webb-telescoop legt zeldzaam moment ster...,"Op een nieuwe, gedetailleerde foto van de Jame...",2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951872/102...,"[nieuw, detailleren, foto, james, webb-ruimtet...","[james, webb-telescoop, legt, zeldzaam, moment..."
22582,https://nos.nl/artikel/2467529-minstens-vijf-d...,Minstens vijf doden door overstromingen in aar...,In Turkije zijn minstens tien doden gevallen d...,2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951932/102...,"[turkije, minstens, dood, geval, overstroming,...","[minstens, dood, overstroming, aardbevingsgebi..."
22583,https://nos.nl/artikel/2467526-politie-pakista...,Politie Pakistan weer slaags met aanhangers bi...,Voor de tweede dag op een rij zijn er hevige o...,2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951853/102...,"[twee, rij, hevig, onlust, huis, pakistaans, o...","[politie, pakistan, slaags, aanhanger, huis, g..."
22584,https://nos.nl/artikel/2467504-wekdienst-15-3-...,Wekdienst 15/3: Stemmen voor Provinciale State...,Goedemorgen! Ruim 13 miljoen Nederlandse kieze...,2023-03-15,"['Binnenland', 'Buitenland']",https://cdn.nos.nl/image/2023/03/15/951807/102...,"[goedemorgen, ruim, nederlands, kiezer, stembu...","[wekdienst, 15/3, stemmen, provinciale, staten..."


In [11]:
#See the most common words, to see whether extra stop words need to be added
articles_exploded = articles.explode('cleaned_article')
count_articles = articles_exploded['cleaned_article'].value_counts()
count_articles[0:30]

titles_exploded = articles.explode('cleaned_title')
count_titles = titles_exploded['cleaned_title'].value_counts()
count_titles[0:30]

dood          1210
wekdienst     1087
vs             943
nieuw          758
trump          549
overlijden     522
groot          512
china          481
brits          466
president      424
duits          413
nederlands     409
rusland        408
politie        400
amerikaans     393
russisch       380
man            377
krijgen        372
premier        371
aanslag        370
protest        361
nederland      351
vrouw          341
oekraïne       336
mens           334
vinden         327
duitsland      318
frans          314
op_pakken      304
terug          303
Name: cleaned_title, dtype: int64

In [12]:
#Function to get the first sentences of an article
def get_summary(text, length):
    text_list = text.split('.')
    summary = ''
    for i in range(length):
        try:
            summary = summary + text_list[i] + '.'
        except:
            break
    return summary

In [13]:
#Get summary by looking at first two sentences of article
#This summary will be displayed in the recommendations
articles['summary'] = articles['article'].apply(lambda row : get_summary(row, 2))
articles

,url,title,article,date,tags,image,cleaned_article,cleaned_title,summary
0,https://nos.nl/artikel/2265855-gewapende-manne...,Gewapende mannen doden 37 burgers in Mali,"In het midden van Mali, in de Mopti-regio, heb...",2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522391/xxl...,"[midden, mali, mopti-regio, wapenen, man, burg...","[gewapend, man, dood, burger, mali]","In het midden van Mali, in de Mopti-regio, heb..."
1,https://nos.nl/artikel/2265853-migranten-ontde...,Migranten ontdekt in matrassen aan Spaanse grens,De Spaanse politie heeft afgelopen weekend twe...,2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522387/xxl...,"[spaans, politie, weekend, afrikaans, migrant,...","[migranten, ontdekken, matrasis, spaans, grens]",De Spaanse politie heeft afgelopen weekend twe...
2,https://nos.nl/artikel/2265844-sea-watch-ziet-...,Sea-Watch ziet situatie op vluchtelingenschip ...,De situatie aan boord van het vluchtelingensch...,2019-01-01,"['Binnenland', 'Buitenland']",https://cdn.nos.nl/image/2018/12/28/521681/xxl...,"[situatie, boord, vluchtelingenschip, sea-watc...","[sea-watch, situatie, vluchtelingenschip, nijpen]",De situatie aan boord van het vluchtelingensch...
3,https://nos.nl/artikel/2265833-bolsonaro-brazi...,Bolsonaro: Brazilië nu bevrijd van socialisme ...,In de Braziliaanse hoofdstad Brasilia is de ui...,2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522367/xxl...,"[braziliaans, hoofdstad, brasilia, uiterst, re...","[bolsonaro, brazilië, bevrijden, socialisme, p...",In de Braziliaanse hoofdstad Brasilia is de ui...
4,https://nos.nl/artikel/2265824-ruimtesonde-new...,Ruimtesonde New Horizons veilig langs ruimtero...,NASA heeft een nieuw signaal binnengekregen va...,2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522361/xxl...,"[nasa, nieuw, signaal, binnengekrijgen, ruimte...","[ruimtesonde, new, horizons, veilig, langs, ru...",NASA heeft een nieuw signaal binnengekregen va...
...,...,...,...,...,...,...,...,...,...
22581,https://nos.nl/artikel/2467536-james-webb-tele...,James Webb-telescoop legt zeldzaam moment ster...,"Op een nieuwe, gedetailleerde foto van de Jame...",2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951872/102...,"[nieuw, detailleren, foto, james, webb-ruimtet...","[james, webb-telescoop, legt, zeldzaam, moment...","Op een nieuwe, gedetailleerde foto van de Jame..."
22582,https://nos.nl/artikel/2467529-minstens-vijf-d...,Minstens vijf doden door overstromingen in aar...,In Turkije zijn minstens tien doden gevallen d...,2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951932/102...,"[turkije, minstens, dood, geval, overstroming,...","[minstens, dood, overstroming, aardbevingsgebi...",In Turkije zijn minstens tien doden gevallen d...
22583,https://nos.nl/artikel/2467526-politie-pakista...,Politie Pakistan weer slaags met aanhangers bi...,Voor de tweede dag op een rij zijn er hevige o...,2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951853/102...,"[twee, rij, hevig, onlust, huis, pakistaans, o...","[politie, pakistan, slaags, aanhanger, huis, g...",Voor de tweede dag op een rij zijn er hevige o...
22584,https://nos.nl/artikel/2467504-wekdienst-15-3-...,Wekdienst 15/3: Stemmen voor Provinciale State...,Goedemorgen! Ruim 13 miljoen Nederlandse kieze...,2023-03-15,"['Binnenland', 'Buitenland']",https://cdn.nos.nl/image/2023/03/15/951807/102...,"[goedemorgen, ruim, nederlands, kiezer, stembu...","[wekdienst, 15/3, stemmen, provinciale, staten...",Goedemorgen! Ruim 13 miljoen Nederlandse kieze...


In [32]:
#Add a datenumber to df
#A number of 0 means it is scraped on 01/01/2019
#A number of 2 means it is scraped on 02/01/2019, etc
#This makes it easy to calculate the difference between dates for articles
articles['datetime_obj'] = articles['date'].apply(lambda row : datetime.strptime(row, '%Y-%m-%d'))
oldestdate = articles['datetime_obj'].loc[0]
articles['datenumber'] = articles['datetime_obj'].apply(lambda row : (row - oldestdate).days)
articles = articles.drop(['datetime_obj'], axis=1)
articles

,url,title,article,date,tags,image,cleaned_article,cleaned_title,summary,datenumber
0,https://nos.nl/artikel/2265855-gewapende-manne...,Gewapende mannen doden 37 burgers in Mali,"In het midden van Mali, in de Mopti-regio, heb...",2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522391/xxl...,"[midden, mali, mopti-regio, wapenen, man, burg...","[gewapend, man, dood, burger, mali]","In het midden van Mali, in de Mopti-regio, heb...",0
1,https://nos.nl/artikel/2265853-migranten-ontde...,Migranten ontdekt in matrassen aan Spaanse grens,De Spaanse politie heeft afgelopen weekend twe...,2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522387/xxl...,"[spaans, politie, weekend, afrikaans, migrant,...","[migranten, ontdekken, matrasis, spaans, grens]",De Spaanse politie heeft afgelopen weekend twe...,0
2,https://nos.nl/artikel/2265844-sea-watch-ziet-...,Sea-Watch ziet situatie op vluchtelingenschip ...,De situatie aan boord van het vluchtelingensch...,2019-01-01,"['Binnenland', 'Buitenland']",https://cdn.nos.nl/image/2018/12/28/521681/xxl...,"[situatie, boord, vluchtelingenschip, sea-watc...","[sea-watch, situatie, vluchtelingenschip, nijpen]",De situatie aan boord van het vluchtelingensch...,0
3,https://nos.nl/artikel/2265833-bolsonaro-brazi...,Bolsonaro: Brazilië nu bevrijd van socialisme ...,In de Braziliaanse hoofdstad Brasilia is de ui...,2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522367/xxl...,"[braziliaans, hoofdstad, brasilia, uiterst, re...","[bolsonaro, brazilië, bevrijden, socialisme, p...",In de Braziliaanse hoofdstad Brasilia is de ui...,0
4,https://nos.nl/artikel/2265824-ruimtesonde-new...,Ruimtesonde New Horizons veilig langs ruimtero...,NASA heeft een nieuw signaal binnengekregen va...,2019-01-01,['Buitenland'],https://cdn.nos.nl/image/2019/01/01/522361/xxl...,"[nasa, nieuw, signaal, binnengekrijgen, ruimte...","[ruimtesonde, new, horizons, veilig, langs, ru...",NASA heeft een nieuw signaal binnengekregen va...,0
...,...,...,...,...,...,...,...,...,...,...
22581,https://nos.nl/artikel/2467536-james-webb-tele...,James Webb-telescoop legt zeldzaam moment ster...,"Op een nieuwe, gedetailleerde foto van de Jame...",2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951872/102...,"[nieuw, detailleren, foto, james, webb-ruimtet...","[james, webb-telescoop, legt, zeldzaam, moment...","Op een nieuwe, gedetailleerde foto van de Jame...",1534
22582,https://nos.nl/artikel/2467529-minstens-vijf-d...,Minstens vijf doden door overstromingen in aar...,In Turkije zijn minstens tien doden gevallen d...,2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951932/102...,"[turkije, minstens, dood, geval, overstroming,...","[minstens, dood, overstroming, aardbevingsgebi...",In Turkije zijn minstens tien doden gevallen d...,1534
22583,https://nos.nl/artikel/2467526-politie-pakista...,Politie Pakistan weer slaags met aanhangers bi...,Voor de tweede dag op een rij zijn er hevige o...,2023-03-15,['Buitenland'],https://cdn.nos.nl/image/2023/03/15/951853/102...,"[twee, rij, hevig, onlust, huis, pakistaans, o...","[politie, pakistan, slaags, aanhanger, huis, g...",Voor de tweede dag op een rij zijn er hevige o...,1534
22584,https://nos.nl/artikel/2467504-wekdienst-15-3-...,Wekdienst 15/3: Stemmen voor Provinciale State...,Goedemorgen! Ruim 13 miljoen Nederlandse kieze...,2023-03-15,"['Binnenland', 'Buitenland']",https://cdn.nos.nl/image/2023/03/15/951807/102...,"[goedemorgen, ruim, nederlands, kiezer, stembu...","[wekdienst, 15/3, stemmen, provinciale, staten...",Goedemorgen! Ruim 13 miljoen Nederlandse kieze...,1534


In [33]:
#save cleaned data

with open("articles_cleaned.pkl", "wb") as file:
    pickle.dump(articles, file)